# New Relic ML Performance Monitoring- Bring Your Own Data
<b> “ML Performance Monitoring” is a library based on the “newrelic_telemetry_sdk” library that helps the user easily send model data to New Relic,so that they can quickly monitor a simple model, directly from a Jupyter notebook or a cloud service. 
In the following notebook, you will see a various ways to use it.</b> 

Note:
this notebook use the libraries: sklearn, pandas, uuid, numpy

### 1. Import libraries


In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error

from new_relic_ml_performance_monitoring.monitor import (
    MLPerformanceMonitoring,
    wrap_model,
)

### 2. Load the iris dataset and split it into train and test sets



In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

boston_dataset = load_boston()
X, y, features_columns, labels_columns = (
    boston_dataset["data"],
    boston_dataset["target"],
    list(boston_dataset["feature_names"]),
    ["target"],
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
)

X_train[:5], y_train[:5]

### 3. Fitting Random Forest Classification to the Training set




In [5]:
xg_reg = xgb.XGBRegressor(
    objective="reg:linear",
    colsample_bytree=0.3,
    learning_rate=0.1,
    max_depth=5,
    alpha=10,
    n_estimators=10,
)
xg_reg.fit(X_train, y_train)

[09:23:54] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=16,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=0, reg_alpha=10, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

### 4. Predicting the Test set results

In [6]:
y_pred = xg_reg.predict(X_test)
y_pred

array([ 9.107417 , 17.003168 , 23.554592 , 11.127839 , 18.847708 ,
       14.83594  , 17.920063 ,  7.8613553, 12.147531 , 18.064953 ,
       18.074614 , 14.411147 , 11.660895 , 15.656607 , 14.011221 ,
       14.031893 , 14.247164 , 23.057888 , 13.79595  , 11.450888 ,
       12.218057 , 15.310874 , 20.628437 , 23.554592 , 15.16144  ,
       13.139177 , 11.691522 , 15.382896 , 15.382896 , 11.007488 ,
       14.734173 , 18.862646 ,  8.471523 , 15.349405 , 15.463427 ,
       19.77869  , 15.875313 ,  9.95701  , 12.919248 , 23.60624  ,
       18.295853 , 12.589657 , 14.179869 , 21.97658  , 12.664099 ,
       16.489174 , 14.038502 , 15.660043 , 12.919248 , 14.453838 ,
       18.862646 , 17.252022 , 14.038502 ,  7.9073224, 14.332668 ,
       10.871179 , 10.826269 ,  8.5319   , 19.534363 ,  8.196141 ,
       12.403829 , 14.235118 , 10.42806  , 13.162136 , 14.247164 ,
       15.931118 , 16.565668 ,  9.804545 , 14.915205 , 18.295853 ,
       13.15519  , 15.318869 , 12.919248 , 16.430174 , 11.8543

### 5. Record inference data to New Relic
<b>   The MLPerformanceMonitoring object requires few parameters:<br> 1. model_name <br> 2. new relic insert key-https://docs.newrelic.com/docs/apis/intro-apis/new-relic-api-keys/#insights-insert-key <br>
##### Optional parameters:<br> 3. metadata dictonrary that will be added to each event (row) of the data<br>4. send_data_metrics- send datafame sammary to New Relic. False as defualt.  <br>5. features_columns- list of the features names in the same order as X<br>6.labels_columns- list of the labels names in the same order as y </b> 


In [7]:
metadata = {"environment": "aws", "dataset": "Boston housing prices", "version": "1.0"}
monitor = wrap_model(
    insert_key=insert_key,
    model=xg_reg,
    staging=True,
    model_name="Boston XGBoost regression",  # the model_name value must be unique per model
    metadata=metadata,
    send_data_metrics=True,
    features_columns=features_columns,
    labels_columns=labels_columns,
)

<b>   You can use the MLPerformanceMonitoring object in vairios ways:
<br> 5.1.  Send your features and prediction as np.array. <br> In this case, the feature columns and the label columns  in new relic will be start with the prefix "feature_" and "lablel_" with numbers, respectively.</b> 

In [ ]:
monitor.record_inference_data(X=X_test, y=y_pred)

<b> 5.2.  Send your features and prediction as pd.DataFrame. <br> In this case, the feature columns and the label columns in new relic will be the DataFrame columns names and will be start with the prefix "feature_" and "lablel_", respectively. <br> The paramter "inference_identifier" can be use of setting a unique inference_identifier for each event(row). Just set the relevent column name in the X DataFrame that need to be used as inference_identifier and this column will be name "inference_identifier" in New Relic. </b>

In [12]:
X_df = pd.DataFrame(
    list(map(np.ravel, X_test)),
    columns=features_columns,
)

y_pred_df = pd.DataFrame(
    list(map(np.ravel, y_pred)),
    columns=labels_columns,
)
X_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,51.13580,0.0,18.10,0.0,0.5970,5.757,100.0,1.4130,24.0,666.0,20.2,2.60,10.11
1,0.05735,0.0,4.49,0.0,0.4490,6.630,56.1,4.4377,3.0,247.0,18.5,392.30,6.53
2,0.03578,20.0,3.33,0.0,0.4429,7.820,64.5,4.6947,5.0,216.0,14.9,387.31,3.76
3,12.04820,0.0,18.10,0.0,0.6140,5.648,87.6,1.9512,24.0,666.0,20.2,291.55,14.10
4,0.03150,95.0,1.47,0.0,0.4030,6.975,15.3,7.6534,3.0,402.0,17.0,396.90,4.56


In [13]:
y_pred_df.head()

,target
0,9.107417
1,17.003168
2,23.554592
3,11.127839
4,18.847708


In [ ]:
import uuid

X_df["uuid"] = X_df.apply(lambda _: str(uuid.uuid4()), axis=1)

monitor.record_inference_data(X=X_df, y=y_pred_df, inference_identifier="uuid")

X_df.head()

<b> 5.3.  Use wrap_model() function to send your model or pipelin as parameter and use them as usual (fit, predict, ect.). This function will send your inference data and data_metrics automaticlly. </b>

In [ ]:
metadata = {"environment": "aws", "dataset": "Boston housing prices", "version": "1.0"}
model = wrap_model(
    insert_key=insert_key,
    model=xg_reg,
    staging=True,
    model_name="Boston XGBoost regression",
    metadata=metadata,
    send_data_metrics=True,
    features_columns=features_columns,
    labels_columns=labels_columns,
)


y_pred = model.predict(
    X=X_df,
    inference_identifier="uuid",
)

### 6. Record metrics to New Relic

<b>Send your model metrics as a dictionary to new relic. You can send new metadata or the fuction use the metadata you set in the object creation. Also, a boolean parameter named "data_metric" can be used to idenify is those metrics are data metric (like mean and std of each feature) or model metrics (like accuracy and f1 score)</b>

In [17]:
rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)), 3)
print(f"RMSE: {rmse}")

RMSE: 10.517


In [19]:
metrics = {
    "RMSE": rmse,
}
model.record_metrics(metrics=metrics, data_metric=False)

model_metric sent successfully
